In [1]:
import nltk
import string
from datasets import load_dataset
import re
from nltk.corpus import stopwords
from nltk import pos_tag
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from scipy.sparse import csr_matrix
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import mlflow
from sklearnex import patch_sklearn
from warnings import filterwarnings
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [3]:
filterwarnings("ignore")

In [4]:
ag_news_dataset = load_dataset("ag_news")
stop_words = set(stopwords.words("english"))

#### Нужно: (ничего, Стемминг, Лемматизация) * (binary, count, tf-idf)) -> F1_Macro

In [5]:
text_preprocess_types = [None, 'стемминг', 'лемматизация']
words_classes = ['N', 'NJ', 'NJV', 'ALL']

#frequency_filtration_types = [None, 'low', 'high', 'both']
frequency_filtration_types = [None, 'low']

vector_representation_types = ['binary', 'count', 'tfidf']

In [6]:
iterations_num = len(text_preprocess_types) * len(words_classes) * len(frequency_filtration_types)
print(iterations_num)

24


In [7]:
def base_ag_news_preprocess(text):
    tokens = text.lower()

    # Удаление спец слов
    special_words = ['reuters', 'afp', 'ap', 'usatoday.com', 'forbes.com', 'target=/stocks/quickinfo/fullquote"' ]
    for word in special_words:
        tokens = tokens.replace(word, '')
    
    pattern = r'[&lt][^<>]*&gt'
    tokens = re.sub(pattern, '', tokens)
    
    # Удаление пунктуации и цифр
    #tokens = ''.join(i if i not in set(string.punctuation) - set('-') | set(string.digits) else ' ' for i in tokens)
    tokens = ''.join(i if i not in set(string.punctuation)  | set(string.digits) else ' ' for i in tokens)
    
    # Токенизация
    tokens = nltk.word_tokenize(tokens)
    
    # Удаление стоп слов
    #stop_wordsL = stop_words - {'no','not'}
    stop_wordsL = stop_words
    tokens = [word for word in tokens if (word not in stop_wordsL and word != '-')]
    return tokens

In [8]:
def different_ag_news_preprocess(tokens, preprocess_type, words_class):
    
    # Обработка слов
    if preprocess_type == 'лемматизация':
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    elif preprocess_type == 'стемминг':
        stemmer = nltk.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    
    # Обработка частей речи
    tokens = pos_tag(tokens)
    if words_class == 'N':
        tokens = [word for word, tag in tokens if tag.startswith('N')]
    elif words_class == 'NJ':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J')]
    elif words_class == 'NJV':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J') or tag.startswith('V')]
    elif words_class == 'ALL':
        tokens = [word for word, _ in tokens]
    
    return tokens  

In [9]:
def frequency_filtration(words_dictionary, frequency_filtration_type):
    if frequency_filtration_type == 'low':
        return dict([(key, value) for key, value in words_dictionary.items() if value >= 10 ])
    elif frequency_filtration_type == 'high':
        return dict([(key, value) for key, value in words_dictionary.items() if value <= 3000])
    elif frequency_filtration_type == 'both':
        return dict([(key, value) for key, value in words_dictionary.items() if 10 <= value <= 3000])
    else:
        return words_dictionary

In [10]:
def dummy(doc):
    return doc

In [18]:
def final_ag_news_preprocess(dataset):
    # Подготовка данных
    x_train = dataset['train']['text']
    y_train = dataset['train']['label']
    
    x_test = dataset['test']['text']
    y_test = dataset['test']['label']
    
    # Базовая обработка
    for i, text in enumerate(x_train):
        x_train[i] = base_ag_news_preprocess(text)
        
    for i, text in enumerate(x_test):
        x_test[i] = base_ag_news_preprocess(text)
    
    index = 2
    # Вариативная обработка
    for preprocess_type in text_preprocess_types: # 3 варианта
        for words_class in words_classes:         # 4 варианта
            if preprocess_type is None and words_class == 'N':
                continue
            words = {}
            xtr = x_train
            xte = x_test
            
            # Обработка текстов
            for i, tokens in enumerate(xtr):
                final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
                xtr[i] = final_tokens
                
                # Заполнение словаря
                for token in final_tokens:
                    if token not in words:
                        words[token] = 1
                    else:
                        words[token] += 1
            
            xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
            # Фильтрация по частоте
            for frequency_filtration_type in frequency_filtration_types: # 4 варианта
                filtered_words = frequency_filtration(words, frequency_filtration_type)
                token_length = len(filtered_words)
                
                # Векторизация слов
                word_list = sorted(filtered_words.keys())
                # Присвоение словам индексов
                words_indexed = {}
                for idx, word in enumerate(word_list):
                    words_indexed[word] = idx
                
                # OHE
                vectorizer_OHE = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8, binary=True)
                x_train_OHE = vectorizer_OHE.fit_transform(xtr)
                x_test_OHE = vectorizer_OHE.transform(xte)
                
                # COUNT
                vectorizer_COUNT = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8)
                x_train_COUNT = vectorizer_COUNT.fit_transform(xtr)
                x_test_COUNT = vectorizer_COUNT.transform(xte)
                
                # TF-IDF
                vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
                x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
                x_test_TFIDF = vectorizer_TFIDF.transform(xte)


                # Построение классификаторов
                clf_OHE = DecisionTreeClassifier(max_depth=750)
                clf_COUNT = DecisionTreeClassifier(max_depth=750)
                clf_TFIDF = DecisionTreeClassifier(max_depth=750)
                
                # Обучение классификаторов
                clf_OHE = clf_OHE.fit(x_train_OHE, y_train)
                clf_COUNT = clf_COUNT.fit(x_train_COUNT, y_train)
                clf_TFIDF = clf_TFIDF.fit(x_train_TFIDF, y_train)
                
                # Тестирование
                predictions_OHE = clf_OHE.predict(x_test_OHE)
                predictions_COUNT = clf_COUNT.predict(x_test_COUNT)
                predictions_TFIDF = clf_TFIDF.predict(x_test_TFIDF)
                
                # Метрики
                macro_score_OHE = f1_score(y_test, predictions_OHE, average='macro')
                macro_score_COUNT = f1_score(y_test, predictions_COUNT, average='macro')
                macro_score_TFIDF = f1_score(y_test, predictions_TFIDF, average='macro')
                
                mlflow.start_run(run_name=f'{preprocess_type}_{words_class}_{frequency_filtration_type}_OHE')
                mlflow.log_param('model', clf_OHE.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                mlflow.sklearn.log_model(clf_OHE, 'DecisionTreeClassifier')
                mlflow.log_metric('macro_score', macro_score_OHE)
                mlflow.end_run()
                
                mlflow.start_run(run_name=f'{preprocess_type}_{words_class}_{frequency_filtration_type}_COUNT')
                mlflow.log_param('model', clf_COUNT.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                mlflow.sklearn.log_model(clf_COUNT, 'DecisionTreeClassifier')
                mlflow.log_metric('macro_score', macro_score_COUNT)
                mlflow.end_run()
                
                mlflow.start_run(run_name=f'{preprocess_type}_{words_class}_{frequency_filtration_type}_TFIDF')
                mlflow.log_param('model', clf_TFIDF.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                mlflow.sklearn.log_model(clf_TFIDF, 'DecisionTreeClassifier')
                mlflow.log_metric('macro_score', macro_score_TFIDF)
                mlflow.end_run()
                
                index += 1
                print(f'Итерация {index} / {iterations_num}')

In [19]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name="agNewsDT")

<Experiment: artifact_location='mlflow-artifacts:/537408113752698407', creation_time=1741637380932, experiment_id='537408113752698407', last_update_time=1741637380932, lifecycle_stage='active', name='agNewsDT', tags={}>

In [20]:
final_ag_news_preprocess(ag_news_dataset)

2025/03/10 23:58:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run None_NJ_None_OHE at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/a9998c497dff418180ac4f9432f2fad3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407


2025/03/10 23:58:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run None_NJ_None_COUNT at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/984bc71c2d134d9b89bb98f83f0045a3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407


2025/03/10 23:58:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run лемматизация_ALL_low_OHE at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/5aec787fc3c047e982feb8517c51e530
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407


2025/03/11 00:58:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run лемматизация_ALL_low_COUNT at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/b32e5a3fd2eb41859eafe27d8f7dcc54
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407


2025/03/11 00:58:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run лемматизация_ALL_low_TFIDF at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/681722f6b7e0448fabb0a34b9f7f0111
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407
Итерация 24 / 24


In [16]:
cc = DecisionTreeClassifier()